In [139]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [140]:
train=pd.read_csv('Train.csv', delimiter=',')
test=pd.read_csv('Test.csv', delimiter=',')

In [141]:
from sklearn.feature_extraction.text import CountVectorizer

In [142]:
cv=CountVectorizer()

In [143]:
X = train['NewsText']

In [144]:
def transform(data, details):
    # 0 == -1 , 1 == 0, 2 == 1
    
    x = []
    if details == 'encode':
        for v, i in enumerate(data):
            if i == -1: x.append(0)
            elif i == 0: x.append(1)
            else: x.append(2)
    else:
        for v, i in enumerate(data):
            if i == 0: x.append(-1)
            elif i == 1: x.append(0)
            else: x.append(1)
                           
    return np.array(x)

In [145]:
y = transform(train['label'].values, details = 'encode')

In [146]:
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold,cross_val_predict, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV, RandomizedSearchCV

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify= y, test_size=0.2, random_state=42)

In [148]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [149]:
rfc = RandomForestClassifier()
scaler = StandardScaler()

In [150]:
cv_train =cv.fit_transform(X_train).toarray()

In [151]:
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
str_kf = StratifiedKFold(n_splits=5) 

In [152]:
cv_results = cross_validate (rfc, cv_train, y_train, cv = skf, scoring = 'accuracy', return_train_score=True)

In [153]:
print(f"Train Score: {cv_results['train_score'].mean()}\n")
print(f"Test Score: {cv_results['test_score'].mean()}")

Train Score: 1.0

Test Score: 0.5849752586594691


In [ ]:
Train Score: 1.0

Test Score: 0.5795321637426902
    
    
Train Score: 1.0

Test Score: 0.5914080071974809

In [49]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [117]:
model = RandomForestClassifier()
param_grid = { 
    'n_estimators': [i for i in range(0,100, 10)],
    'min_samples_leaf':[1,2,5,10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy'],
    'min_samples_split':[2, 5, 10, 15, 100],
    'min_samples_leaf':[1, 2, 5, 10] 
}



gsearch_rfc = GridSearchCV(estimator = model,param_grid = param_grid, scoring='accuracy',n_jobs=-1, cv=str_kf, verbose = 1)

gsearch_rfc.fit(cv_train, y_train)

print(f'Best: {gsearch_rfc.best_score_}, using {gsearch_rfc.best_params_}')
results_df = pd.DataFrame(gsearch_rfc.cv_results_)
results_df.head()

Fitting 5 folds for each of 6000 candidates, totalling 30000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.46842105 0.48502024 ... 0.36464238 0.36464238 0.36464238]
  warnings.warn(


Best: 0.6045883940620783, using {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 90}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003234,0.003510,0.000000,0.000000,gini,4,auto,1,2,0,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000
1,0.042747,0.003933,0.000000,0.000000,gini,4,auto,1,2,10,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.564103,0.435897,0.473684,0.500000,0.368421,0.468421,0.065173,2093
2,0.076508,0.004215,0.004025,0.004217,gini,4,auto,1,2,20,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.410256,0.435897,0.500000,0.552632,0.526316,0.485020,0.053858,1593
3,0.106694,0.004232,0.006449,0.004484,gini,4,auto,1,2,30,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.384615,0.435897,0.552632,0.500000,0.473684,0.469366,0.056909,1984
4,0.140596,0.001528,0.010082,0.000030,gini,4,auto,1,2,40,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.435897,0.487179,0.578947,0.631579,0.605263,0.547773,0.074146,188


In [136]:
tune_rfc = gsearch_rfc.best_estimator_

In [135]:
gsearch_rfc.best_score_

0.6045883940620783

In [120]:
gsearch_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 90}

In [103]:
tune_rfc = RandomForestClassifier(criterion='entropy', max_depth=8, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators = 90)

#tune_rfc = RandomForestClassifier(criterion='entropy', max_depth=6, max_features='sqrt', min_samples_leaf=1, min_samples_split=5)

In [123]:
scaled_val = scaler.transform(cv.transform(X_test).toarray())

In [124]:
scaled_test = scaler.transform(cv.transform(test['NewsText']).toarray())

In [138]:
scaled_val

array([[-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746],
       [-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746],
       [-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746],
       ...,
       [-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746],
       [-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746],
       [-0.11867817, -0.07235746, -0.10259784, ..., -0.07235746,
        -0.14586499, -0.07235746]])

In [126]:
cv_score=[]
test_pred=[]
train_score=[]
cv_model_store=[]# - uo

sub_pred = []

fold= skf
X = pd.DataFrame(cv_train)
yy = y_train
Xtest = pd.DataFrame(scaled_val) 
yytest = y_test

ttest = scaled_test

for train_index, test_index in fold.split(X,pd.Series(yy)):
    X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
    y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]
    
    tune_rfc.fit(X_trainx,y_trainx)
    cv_model_store.append(tune_rfc)
    train_preds=tune_rfc.predict(X_trainx)
    print("Train Accuracy: ",accuracy_score(y_trainx,train_preds))
    train_score.append(accuracy_score(y_trainx,train_preds))
    
    val_preds=tune_rfc.predict(X_val)
    print("Validation Accuracy: ",accuracy_score(y_val,val_preds))
    cv_score.append(accuracy_score(y_val,val_preds))
    
    test_p = tune_rfc.predict(Xtest)
    print("Test Accuracy Score : ",accuracy_score(yytest ,test_p))
    print("______________\n")
    test_pred.append(accuracy_score(yytest ,test_p))
    
    tested = tune_rfc.predict_proba(ttest)
    print("Test has been Scored")
    print("______________\n")
    sub_pred.append(tested)

print(f'Training Mean ROC AUC Score {np.mean(train_score)}')
print(f'Validation Mean ROC AUC Score {np.mean(cv_score)}')
print(f'Test Mean ROC AUC Score {np.mean(test_pred)}')

Train Accuracy:  0.8431372549019608
Validation Accuracy:  0.38461538461538464
Test Accuracy Score :  0.5416666666666666
______________

Test has been Scored
______________

Train Accuracy:  0.8300653594771242
Validation Accuracy:  0.5897435897435898
Test Accuracy Score :  0.5208333333333334
______________

Test has been Scored
______________

Train Accuracy:  0.8246753246753247
Validation Accuracy:  0.42105263157894735
Test Accuracy Score :  0.5625
______________

Test has been Scored
______________

Train Accuracy:  0.8051948051948052
Validation Accuracy:  0.47368421052631576
Test Accuracy Score :  0.4583333333333333
______________

Test has been Scored
______________

Train Accuracy:  0.8636363636363636
Validation Accuracy:  0.6578947368421053
Test Accuracy Score :  0.5
______________

Test has been Scored
______________

Train Accuracy:  0.7908496732026143
Validation Accuracy:  0.5384615384615384
Test Accuracy Score :  0.5625
______________

Test has been Scored
______________

Trai

In [127]:
value = (np.sum(sub_pred, axis = 0))/15

In [128]:
val = np.argmax(value, axis = 1)

In [129]:
pred = transform(val, details = 'decode')

In [130]:
pd.Series(pred).value_counts()

 1    37
-1    21
 0     2
dtype: int64

In [131]:
test['label'] = pred

In [132]:
sub = test[['id', 'label']]

In [133]:
sub['label'].value_counts()

 1    37
-1    21
 0     2
Name: label, dtype: int64

In [134]:
sub.to_csv('new_model.csv', index = False)